#Part 0 - Installation

In [ ]:
!pip install tensorflow==1.15.0
!pip install keras==2.1.4

#Part 1 : Building a CNN

In [ ]:
#import Keras packages
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
import numpy as np

In [ ]:
# Initializing the CNN
from keras import applications
from keras.models import Sequential, Model 

np.random.seed(1337)

model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (128, 128, 3))

x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(512, activation="relu")(x)
predictions = Dense(4, activation="softmax")(x)

# creating the final model 
classifier = Model(input = model.input, output = predictions)


In [ ]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
print(classifier.summary())

#Part 2 - Fitting the data set

In [ ]:
!unzip "/content/data.zip"

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(
        '/content/data/train',
        target_size=(128, 128),
        batch_size=16,
        class_mode='categorical' )
label_map = (training_set.class_indices)

test_set = test_datagen.flow_from_directory(
        '/content/data/val',
        target_size=(128, 128),
        batch_size=16,
        class_mode='categorical')

In [ ]:
history = classifier.fit_generator(
        training_set,
        steps_per_epoch=20,
        epochs=20,        
        validation_data=test_set,
        validation_steps=100)

In [ ]:
history

#Part 3 - Save Model

In [ ]:
import h5py

classifier.save('saved_model')
print('Saved trained model as %s ' % 'saved_model.h5')

#Part 4 - Visualization

In [ ]:
import matplotlib.pyplot as plt

# Plot history: MAE
plt.plot(history.history['val_acc'], label='Val Accuracy')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Persormance')
plt.ylabel('value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()